### Install the required libraries

In [2]:
%pip install -U -q google-generativeai
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import the necessary libraries

In [120]:
import google.generativeai as genai
import os
from dotenv import load_dotenv
from IPython.display import display
from IPython.display import Markdown
import textwrap
import time
import re

### Setup the API Key

In [4]:
load_dotenv
api_key = os.environ.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

### Converts a string to Markdown

In [122]:
def to_markdown(text):
    # Replace bullet points (•) with Markdown-compatible lists (* item)
    text = text.replace('•', '  * ')

    # Function to preserve code blocks
    def preserve_code(match):
        return f"\n```python\n{match.group(1)}\n```\n"  # Ensures correct markdown

    # Extract and preserve Python code blocks
    text = re.sub(r"```python\n(.*?)\n```", preserve_code, text, flags=re.DOTALL)

    # Split text into lines for better processing
    lines = text.split("\n")
    formatted_lines = []
    inside_code_block = False

    for line in lines:
        # Detect start and end of a code block
        if line.startswith("```"):
            inside_code_block = not inside_code_block
            formatted_lines.append(line)
            continue

        # Apply blockquote formatting **only** to non-code lines
        if not inside_code_block:
            line = line.strip()  # Remove leading/trailing whitespace from each line
            if line:  # Only add non-empty lines
                formatted_lines.append(f"> {line}")
        else:
            formatted_lines.append(line)

    # Join lines back into a full formatted text
    formatted_text = "\n".join(formatted_lines)
    
    return Markdown(formatted_text)

### Define the system instructions

In [123]:
SYSTEM_INSTRUCTIONS = """
You are an expert Python developer and AI assistant.  You specialize in helping users create, understand, 
and improve Python code.  You are knowledgeable in various aspects of Python development, including:

*   **Code Generation:** You can write Python code to solve specific problems or implement desired functionalities. 
    You adhere to Pythonic style (PEP 8) and strive to produce clean, efficient, well-documented, and testable code. 
    You consider appropriate design patterns when generating code.

*   **Code Explanation:** You can clearly and concisely explain existing Python code, 
    breaking down complex logic into understandable parts. You provide insights into the code's purpose,
    functionality, potential issues, and how it relates to design patterns or SOLID principles (if applicable).

*   **Best Practices:** You are well-versed in Python best practices, including coding style (PEP 8),
    code organization, error handling, testing, and performance optimization. 
    You advise users on how to write more maintainable, robust, and efficient Python code.

*   **Software Architecture:** You can help users design the architecture of Python applications,
    offering guidance on choosing appropriate design patterns (e.g., Factory, Singleton, Observer, Strategy),
    modules, and frameworks. You are familiar with SOLID principles (Single Responsibility, Open/Closed, 
    Liskov Substitution, Interface Segregation, Dependency Inversion) and can explain how they apply to Python code.
    You discuss trade-offs and help users make informed decisions.

*   **Debugging:** You can assist users in identifying and fixing bugs in their Python code. 
    You suggest debugging strategies, help users interpret error messages, and encourage the use of unit tests.

*   **General Python Knowledge:** You possess a broad understanding of the Python language, its standard library,
    and popular third-party libraries. You answer questions about Python syntax, semantics, and usage.

When providing code, always adhere to PEP 8 style guidelines. Use meaningful variable names 
and add comprehensive docstrings to explain complex logic, function signatures, and class purposes.
If you are unsure about a user's intent, ask clarifying questions before providing a response. 
Be patient and supportive, and strive to help users improve their Python skills.  
When discussing architecture or design, explain how SOLID principles or specific design patterns can be applied 
to solve the problem at hand.  If a user's code violates best practices or SOLID principles, 
gently point out the issues and suggest improvements.
"""

display(to_markdown(SYSTEM_INSTRUCTIONS)) 

> You are an expert Python developer and AI assistant.  You specialize in helping users create, understand,
> and improve Python code.  You are knowledgeable in various aspects of Python development, including:
> *   **Code Generation:** You can write Python code to solve specific problems or implement desired functionalities.
> You adhere to Pythonic style (PEP 8) and strive to produce clean, efficient, well-documented, and testable code.
> You consider appropriate design patterns when generating code.
> *   **Code Explanation:** You can clearly and concisely explain existing Python code,
> breaking down complex logic into understandable parts. You provide insights into the code's purpose,
> functionality, potential issues, and how it relates to design patterns or SOLID principles (if applicable).
> *   **Best Practices:** You are well-versed in Python best practices, including coding style (PEP 8),
> code organization, error handling, testing, and performance optimization.
> You advise users on how to write more maintainable, robust, and efficient Python code.
> *   **Software Architecture:** You can help users design the architecture of Python applications,
> offering guidance on choosing appropriate design patterns (e.g., Factory, Singleton, Observer, Strategy),
> modules, and frameworks. You are familiar with SOLID principles (Single Responsibility, Open/Closed,
> Liskov Substitution, Interface Segregation, Dependency Inversion) and can explain how they apply to Python code.
> You discuss trade-offs and help users make informed decisions.
> *   **Debugging:** You can assist users in identifying and fixing bugs in their Python code.
> You suggest debugging strategies, help users interpret error messages, and encourage the use of unit tests.
> *   **General Python Knowledge:** You possess a broad understanding of the Python language, its standard library,
> and popular third-party libraries. You answer questions about Python syntax, semantics, and usage.
> When providing code, always adhere to PEP 8 style guidelines. Use meaningful variable names
> and add comprehensive docstrings to explain complex logic, function signatures, and class purposes.
> If you are unsure about a user's intent, ask clarifying questions before providing a response.
> Be patient and supportive, and strive to help users improve their Python skills.
> When discussing architecture or design, explain how SOLID principles or specific design patterns can be applied
> to solve the problem at hand.  If a user's code violates best practices or SOLID principles,
> gently point out the issues and suggest improvements.

### Define the generation config

In [13]:
GENERATION_CONFIG = genai.types.GenerationConfig(
    temperature = 0.7,  
    top_p = 0.95,  
    top_k = 40,  
)

### Select the model, pass the system instructions and the generation config

In [16]:
model = genai.GenerativeModel(
    model_name = 'gemini-1.5-pro',
    system_instruction = SYSTEM_INSTRUCTIONS,
    generation_config = GENERATION_CONFIG
)

### Welcome text

In [124]:
def welcome_text():
    text = """
### Welcome to the Python Chatbot! 🚀  
Feel free to ask anything about Python, from basic concepts to best practices.<br><br>  
I'm here to help! 😊<br><br>  
To quit the chat, enter **quit** ❌  
"""
    return display(to_markdown(text))

### Ending conversation text

In [125]:
def ending_conversation_text():
    display(to_markdown('### Ending Conversation ... 🚫'))
    time.sleep(2)
    display(to_markdown('### Talk to you next time! 👋'))

### Display response

In [128]:
def response_text(chat_session):
    user_text = f"#### {chat_session.history[-2].role.capitalize()}: {chat_session.history[-2].parts[0].text}"
    model_text = f"{chat_session.history[-1].role.capitalize()}: {chat_session.history[-1].parts[0].text}"
    display(to_markdown(user_text))
    display(to_markdown(model_text))

### Chatbot logic

In [131]:
chat_session = model.start_chat(history=[])
welcome_text()
prompt = input("User:").strip() 

while prompt != "quit":
    if prompt.lower() == "quit" or prompt == "":  # Exit on "quit" or empty input (Enter key)
        break
    chat_session.send_message(prompt)
    response_text(chat_session)
    #response_texts(chat_session)
    prompt = input('User:').strip()
ending_conversation_text()    

> ### Welcome to the Python Chatbot! 🚀
> Feel free to ask anything about Python, from basic concepts to best practices.<br><br>
> I'm here to help! 😊<br><br>
> To quit the chat, enter **quit** ❌

> #### User: What is python?

> Model: Python is a high-level, general-purpose programming language known for its readability and versatility.  Here's a breakdown of its key characteristics:
> **Key Features:**
> * **Interpreted:**  Python code is executed line by line by an interpreter, making it easier to debug and test.  You don't need a separate compilation step like in languages like C or C++.
> * **Dynamically Typed:**  You don't need to explicitly declare variable types.  Python infers the type at runtime. This simplifies code but requires careful testing.
> * **Object-Oriented:** Python supports object-oriented programming (OOP) principles like encapsulation, inheritance, and polymorphism. This allows for modular, reusable code.
> * **Large Standard Library:** Python comes with a vast collection of modules (pre-written code) for various tasks, from web development to data science.  This reduces development time.
> * **Extensive Third-Party Libraries:**  A thriving community contributes to a huge ecosystem of libraries for specialized needs (e.g., NumPy for numerical computing, Pandas for data analysis, Django for web frameworks).
> * **Cross-Platform:** Python code can run on various operating systems (Windows, macOS, Linux) with minimal or no modification.
> * **Readability:** Python emphasizes clear syntax, using indentation to define code blocks.  This makes it easier to read and understand, particularly for beginners.
> **Common Use Cases:**
> * **Web Development:** Building web applications using frameworks like Django and Flask.
> * **Data Science and Machine Learning:** Analyzing data, creating machine learning models, and visualizing results using libraries like NumPy, Pandas, Scikit-learn, and TensorFlow/PyTorch.
> * **Scripting and Automation:** Automating tasks, system administration, and scripting for various applications.
> * **Desktop Applications:** Creating GUI applications using libraries like Tkinter, PyQt, or Kivy.
> * **Game Development:** Developing simple games using libraries like Pygame.
> * **Embedded Systems:**  Python is increasingly used in embedded systems, particularly with microcontrollers like the Raspberry Pi.
> **Example:**
```python
def greet(name):
    """Greets the person passed in as a parameter."""  # Docstring explains the function
    print(f"Hello, {name}!")

greet("World")  # Output: Hello, World!
```
> This simple example demonstrates Python's clear syntax and how easy it is to get started.  If you have any specific areas you'd like to explore further, just let me know!

> #### User: How about data structure in python?

> Model: Python offers several built-in data structures that are fundamental for organizing and manipulating data. Here's an overview of the most commonly used ones:
> **1. Lists:**
> * **Mutable:** Lists can be modified after creation (adding, removing, or changing elements).
> * **Ordered:** Elements maintain their order of insertion.
> * **Allow Duplicates:** Lists can contain multiple instances of the same value.
> * **Versatile:** Can store elements of different data types.
```python
my_list = [1, 2, "hello", 3.14, True]
my_list.append(5)  # Add an element to the end
my_list[0] = 0     # Modify an element
```
> **2. Tuples:**
> * **Immutable:** Once created, tuples cannot be changed.
> * **Ordered:** Elements maintain their order.
> * **Allow Duplicates:** Can contain multiple instances of the same value.
```python
my_tuple = (1, 2, "hello", 3.14, True)
# my_tuple[0] = 0  # This would raise an error because tuples are immutable
```
> **3. Sets:**
> * **Mutable:** Sets can be modified.
> * **Unordered:** Elements do not have a specific order.
> * **No Duplicates:** Sets automatically remove duplicate values.
> * **Efficient for Membership Testing:** Checking if an element exists in a set is very fast.
```python
my_set = {1, 2, 3, 2, 1}  # Duplicates are removed
print(my_set)            # Output: {1, 2, 3}
my_set.add(4)
my_set.remove(1)
```
> **4. Dictionaries:**
> * **Mutable:** Dictionaries can be changed.
> * **Unordered (before Python 3.7):**  In older versions, dictionaries did not maintain order.
> * **Ordered (Python 3.7+):** From Python 3.7 onwards, dictionaries preserve insertion order.
> * **Store Key-Value Pairs:** Each element is a pair of a unique key and its associated value.
> * **Efficient for Lookups:** Retrieving values based on their keys is very fast.
```python
my_dict = {"name": "Alice", "age": 30, "city": "New York"}
print(my_dict["name"])  # Output: Alice
my_dict["age"] = 31     # Modify a value
my_dict["country"] = "USA" # Add a new key-value pair
```
> **Choosing the Right Data Structure:**
> The best data structure depends on the specific needs of your program:
> * **Lists:** When you need an ordered collection that can be modified.
> * **Tuples:** When you need an immutable ordered collection (e.g., representing a record).
> * **Sets:** When you need a collection of unique items and membership testing is important.
> * **Dictionaries:** When you need to store and retrieve data based on keys.
> Could you tell me more about what you'd like to do with data structures in Python?  Knowing your specific use case will help me provide more tailored advice. For example, are you interested in algorithms that work with these data structures, or are you building a specific application?

> #### User: what does my_dict = {"name": "Alice", "age": 30, "city": "New York"}  mean?

> Model: That line of code creates a Python dictionary and assigns it to the variable `my_dict`. Let's break it down:
> * **`my_dict`:** This is the variable name. You can choose any valid variable name.  It now refers to the dictionary you're creating.
> * **`=`:** The assignment operator. It assigns the value on the right-hand side to the variable on the left-hand side.
> * **`{...}`:** Curly braces define a dictionary literal.  Dictionaries store data in key-value pairs.
> * **`"name": "Alice"`:** This is a key-value pair.
> * `"name"` is the key (a string).  Keys must be immutable (e.g., strings, numbers, tuples).
> * `"Alice"` is the value associated with the key "name". Values can be of any data type.
> * **`"age": 30`:** Another key-value pair. The key is the string `"age"`, and the value is the integer `30`.
> * **`"city": "New York"`:**  A third key-value pair.  The key is `"city"`, and the value is `"New York"`.
> **In essence, this dictionary represents information about a person:**
> * Their name is Alice.
> * Their age is 30.
> * They live in New York City.
> **Accessing Values:**
> You can access the values in a dictionary using their corresponding keys:
```python
print(my_dict["name"])  # Output: Alice
print(my_dict["age"])   # Output: 30
print(my_dict["city"])  # Output: New York
```
> **Modifying and Adding Entries:**
> Dictionaries are mutable, so you can change existing values or add new key-value pairs:
```python
my_dict["age"] = 31       # Update Alice's age
my_dict["country"] = "USA"  # Add a new key-value pair
print(my_dict) # Output: {'name': 'Alice', 'age': 31, 'city': 'New York', 'country': 'USA'}
```
> Is there anything else about dictionaries you'd like to know?  Perhaps how to iterate through them, check if a key exists, or use dictionary comprehensions?  I'm happy to explain further.

> ### Ending Conversation ... 🚫

> ### Talk to you next time! 👋